<h1>Modelagem de dados sequenciais usando redes neurais recorrentes</h1>

<p align=center><img src=https://datascience.eu/wp-content/uploads/2020/05/image-513-1024x347.png></p>

Tivemos oportunidade de focar em redes neurais convolucionais (*CNNs*), de forma a cobrir os blocos de construção das arquiteturas *CNN* e como implementar *CNNs* profundas no *TensorFlow*. Por fim, você aprendeu a usar *CNNs* para classificação de imagens.

Aqui, exploraremos as redes neurais recorrentes (*RNNs*) e veremos sua aplicação na modelagem de dados sequenciais.

### Apresentando dados sequenciais
Vamos começar nossa discussão sobre *RNNs* observando a natureza dos dados sequenciais, que são mais comumente conhecidos como dados de sequência ou **sequências**. Vamos dar uma olhada nas propriedades únicas das sequências que as tornam diferentes de outros tipos de dados. Veremos então como podemos representar dados sequenciais e explorar as várias categorias de modelos para dados sequenciais, que são baseados na entrada e saída de um modelo. Isso nos ajudará a explorar a relação entre *RNNs* e sequências.

### Modelagem de dados sequenciais - a ordem importa

O que torna as sequências únicas, em comparação com outros tipos de dados, é que os elementos em uma sequência aparecem em uma determinada ordem e não são independentes uns dos outros. Algoritmos típicos de aprendizado de máquina para aprendizado supervisionado pressupõem que a entrada de dados é **independente e e distribuída de forma idêntica (IID)**, o que significa que os exemplos de treinamento são `mutuamente independentes` e têm a mesma distribuição subjacente.

Nesse sentido, com base na suposição de independência mútua, a ordem em que os exemplos de treinamento são dados ao modelo é **irrelevante**. Por exemplo, se tivermos uma amostra composta por n exemplos de treinamento, $\small x^{(1)}, x^{(2)},\cdots, x^{(n)} $, a ordem em que usamos os dados para treinar nosso algoritmo de aprendizado de máquina não importa. Um exemplo desse cenário seria o conjunto de dados Iris, muito conhecido. No conjunto de dados Iris, cada flor foi medida independentemente e as medidas de uma flor não influenciam as medidas de outra flor.

No entanto, essa suposição não é válida quando lidamos com sequências – por definição, **a ordem é importante**. Prever o valor de mercado de uma determinada ação seria um exemplo desse cenário. Por exemplo, suponha que temos uma amostra de n exemplos de treinamento, onde cada exemplo de treinamento representa o valor de mercado de uma determinada ação em um determinado dia. Se nossa tarefa é prever o valor do mercado de ações para os próximos três dias, faria sentido considerar os preços das ações anteriores em uma ordem de data para derivar tendências, em vez de utilizar esses exemplos de treinamento em uma ordem aleatória.

> #### Dados sequenciais versus dados de séries temporais
> Os dados de série temporal são um tipo especial de dados sequenciais, em que cada exemplo está associado a uma dimensão de tempo. Em dados de séries temporais, as amostras são coletadas em *timestamps* sucessivos e, portanto, a dimensão de tempo determina a ordem entre os pontos de dados. Por exemplo, preços de ações e registros de voz ou fala são dados de séries temporais.
>
> Por outro lado, nem todos os dados sequenciais têm a dimensão temporal, por exemplo, dados de texto ou sequências de DNA, onde os exemplos são ordenados, mas não se qualificam como dados de séries temporais. Como você verá, abordaremos alguns exemplos de Processamento de Linguagem Natural (NLP) e modelagem de texto que não são dados de série temporal, mas observe que as *RNNs* também podem ser usados para dados de série temporal.

### Representando sequências

Estabelecemos que a ordem entre os pontos de dados é importante em dados sequenciais, então precisamos encontrar uma maneira de aproveitar essas informações de pedido em um modelo de aprendizado de máquina. Ao longo das explicações, representaremos as sequências como $\small \left \langle x^{(1)},x^{(2)},\cdots, x^{(T)}  \right \rangle$ . Os índices sobrescritos indicam a ordem das instâncias e o comprimento da sequência é $\small T$. Para um exemplo sensato de sequências, considere dados de séries temporais, onde cada ponto de exemplo, $\small x^{(T)}$, pertence a um determinado tempo, $\small t$. A figura a seguir mostra um exemplo de dados de série temporal em que os recursos de entrada ($\small x$) e os rótulos de destino ($\small y$) seguem naturalmente a ordem de acordo com seu eixo de tempo; portanto, ambos os `x` e `y` são sequências:

![](imagens\sequencias.PNG)

Como já mencionamos, os modelos de rede neural padrão (*RN*) que abordamos até agora, como o *multilayer perceptron* (MLP) e as *CNNs* para dados de imagem, assumem que os exemplos de treinamento são independentes uns dos outros e, portanto, não incorporam **informação de ordenamento**. Podemos dizer que tais modelos não possuem **memória** de exemplos de treinamento vistos anteriormente. Por exemplo, as amostras são passadas pelas etapas de *feedforward* e *backpropagation* e os pesos são atualizados independentemente da ordem em que os exemplos de treinamento são processados. As *RNNs*, por outro lado, são projetadas para modelar sequências e são capazes de lembrar informações passadas e processar novos eventos de acordo, o que é uma clara vantagem ao trabalhar com dados de sequência.

### As diferentes categorias de modelagem de sequência

A modelagem de sequência tem muitas aplicações fascinantes, como tradução de idiomas (por exemplo, tradução de texto de inglês para alemão), legendas de imagens e geração de texto. No entanto, para escolher uma arquitetura e abordagem apropriadas, temos que entender e ser capazes de distinguir entre essas diferentes tarefas de modelagem de sequência. A figura a seguir, baseada nas explicações do excelente artigo The Unreasonable Effectiveness of Recurrent Neural Networks, de Andrej Karpathy (http://karpathy.github.io/2015/05/21/rnn-effectiveness/), resume a sequência mais comum tarefas de modelagem, que dependem das categorias de relacionamento de dados de entrada e saída:

![](imagens\modelagem_de_sequencia.PNG)

Vamos discutir as diferentes categorias de relacionamento entre dados de entrada e saída, que foram descritas na figura anterior, com mais detalhes. Se nem os dados de entrada nem de saída representam sequências, então estamos lidando com dados padrão e podemos simplesmente usar um perceptron multicamada (ou outro modelo de classificação abordado anteriormente) para modelar esses dados. No entanto, se a entrada ou a saída for uma sequência, a tarefa de modelagem provavelmente se enquadra em uma destas categorias:
* **Muitos para um**: Os dados de entrada são uma sequência, mas a saída é um vetor de tamanho fixo ou escalar, não uma sequência. Por exemplo, na análise de sentimentos, a entrada é baseada em texto (por exemplo, uma resenha de filme) e a saída é um rótulo de classe (por exemplo, um rótulo que indica se um revisor gostou do filme).

* **Um para muitos**: Os dados de entrada estão no formato padrão e não em sequência, mas a saída é uma sequência. Um exemplo dessa categoria é a legendagem de imagens — a entrada é uma imagem e a saída é uma frase em inglês que resume o conteúdo dessa imagem.

* **Muitos para muitos**: As matrizes de entrada e saída são sequências. Esta categoria pode ser dividida com base na sincronização da entrada e saída. Um exemplo de uma tarefa de modelagem sincronizada de muitos para muitos é a classificação de vídeo, onde cada quadro em um vídeo é rotulado. Um exemplo de uma tarefa de modelagem muitos-para-muitos *atrasada* seria traduzir uma linguagem para outra. Por exemplo, uma frase inteira em inglês deve ser lida e processada por uma máquina antes que sua tradução para o alemão seja produzida. 

Agora, depois de resumir as três grandes categorias de modelagem de sequência, podemos avançar para discutir a estrutura de uma RNN.

### RNNs para modelagem de sequências 
Nesta seção, antes de começarmos a implementar *RNNs* no *TensorFlow*, discutiremos os principais conceitos de *RNNs*. Começaremos examinando a estrutura típica de uma *RNN*, que inclui um componente recursivo para modelar dados de sequência. Em seguida, examinaremos como as ativações dos neurônios são computadas em uma *RNN* típica. Isso criará um contexto para discutirmos os desafios comuns no treinamento de *RNNs* e, em seguida, discutiremos soluções para esses desafios, como *LSTM* e unidades recorrentes fechadas (*GRUs*).

### Entendendo o mecanismo de loop *RNN*
Vamos começar com a arquitetura de uma *RNN*. A figura a seguir mostra uma *RN* *feedforward* padrão e um *RNN* lado a lado para comparação:

![](imagens\mecanismo_loop_rnn.PNG)

Ambas as redes têm apenas uma camada oculta. Nesta representação, as unidades não são exibidas, mas assumimos que a camada de entrada ($\small x$), a camada oculta ($\small h$) e a camada de saída ($\small o$) são vetores que contêm muitas unidades.

> ##### Determinando o tipo de saída de um RNN
> Essa arquitetura RNN genérica pode corresponder às duas categorias de modelagem de sequência em que a entrada é uma sequência. Normalmente, uma camada recorrente pode retornar uma sequência como saída, $\small \left \langle o^{(1)},o^{(2)},\cdots, o^{(T)}  \right \rangle$, ou simplesmente retornar a última saída (em $\small t = T$, ou seja, $\small o^{(T)}$). Assim, pode ser muitos para muitos ou muitos para um se, por exemplo, usarmos apenas o último elemento, $\small o^{(T)}$, como a saída final.
>
> Como você verá mais tarde, na *API TensorFlow Keras*, o comportamento de uma camada recorrente em relação ao retorno de uma sequência como saída ou simplesmente usar a última saída pode ser especificado definindo o argumento `return_sequences` como `True` ou `False`, respectivamente.

Em uma rede *feedforward* padrão, as informações fluem da entrada para a camada oculta e, em seguida, da camada oculta para a camada de saída. Por outro lado, em uma *RNN*, a camada oculta recebe sua entrada tanto da camada de entrada da etapa de tempo atual quanto da camada oculta da etapa de tempo anterior.

O fluxo de informações em etapas de tempo adjacentes na camada oculta permite que a rede tenha uma memória de eventos passados. Esse fluxo de informações geralmente é exibido como um *loop*, também conhecido como **recurrent edge** (borda recorrente) em notação de gráfico, que é como essa arquitetura *RNN* geral recebeu seu nome.

Semelhante aos *perceptrons* multicamadas, as *RNN*s podem consistir em várias camadas ocultas. Observe que é uma convenção comum se referir às *RNNs* com uma camada oculta como uma *RNN* de camada única, que não deve ser confundido com as RNs de camada única sem uma camada oculta, como *Adaline* ou *regressão logística*. A figura a seguir ilustra uma *RNN* com uma camada oculta (superior) e uma *RNN* com duas camadas ocultas (inferior):

![](imagens\rnn_oculta.PNG)

Para examinar a arquitetura das *RNNs* e o fluxo de informações, pode-se desdobrar uma representação compacta com uma aresta recorrente, que você pode ver na figura anterior.

Como sabemos, cada unidade oculta em uma *RN* padrão recebe apenas uma entrada – a pré-ativação de rede associada à camada de entrada. Em contraste, cada unidade oculta em uma *RNN* recebe dois conjuntos distintos de entrada – a pré-ativação da camada de entrada e a ativação da mesma camada oculta da etapa de tempo anterior, $\small t – 1$.

Na primeira etapa de tempo, $\small t = 0$, as unidades ocultas são inicializadas com zeros ou pequenos valores aleatórios. Então, em um passo de tempo em que $\small t > 0$, as unidades ocultas recebem sua entrada do ponto de dados no momento atual, $\small x^{(T)}$, e os valores anteriores das unidades ocultas em $\small t – 1$, indicados como $\small h^{(t-1)}$.

Da mesma forma, no caso de uma RNN multicamada, podemos resumir o fluxo de informações da seguinte forma:
* $\small layer$ = 1: Aqui, a camada oculta é representada como $h_1^{(t)}$ e recebe sua entrada do ponto de dados, $x^{(t)}$, e os valores ocultos na mesma camada, mas no passo de tempo anterior, $h_1^{(t - 1)}$.
* $\small layer$ = 2: A segunda camada oculta, $h_2^{(t)}$ , recebe suas entradas das saídas da camada abaixo na etapa de tempo atual $(o_1^{(t)})$ e seus próprios valores ocultos da etapa de tempo anterior, $h_2^{(t-1)}$.

Como, neste caso, cada camada recorrente deve receber uma sequência como entrada, todas as camadas recorrentes, exceto a última, devem retornar uma sequência como saída (ou seja, `return_sequences=True`). O comportamento da última camada recorrente depende do tipo de problema.

### Computando ativações em uma *RNN*
Agora que você entende a estrutura e o fluxo geral de informações em uma *RNN*, vamos ser mais específicos e calcular as ativações reais das camadas ocultas, bem como a camada de saída. Para simplificar, consideraremos apenas uma única camada oculta; no entanto, o mesmo conceito se aplica às *RNNs* multicamadas.

Cada aresta direcionada (as conexões entre caixas) na representação de uma *RNN* que acabamos de ver está associada a uma matriz de pesos. Esses pesos não dependem do tempo, $\small t$; portanto, eles são compartilhados ao longo do eixo do tempo. As diferentes matrizes de peso em uma *RNN* de camada única são as seguintes:
* $\small \textbf{W}_{xh}$: A matriz de peso entre a entrada, $\small x^{(t)}$, e a camada oculta, $\small \textbf{h}$
* $\small \textbf{W}_{hh}$: A matriz de peso associada à aresta recorrente
* $\small \textbf{W}_{ho}$: A matriz de peso entre a camada oculta e a camada de saída

Essas matrizes de peso são representadas na figura a seguir:

![](imagens\matriz_pesos.PNG)

Em certas implementações, você pode observar que as matrizes de peso, $\small \textbf{W}_{xh}$ e $\small \textbf{W}_{hh}$, são concatenadas a uma matriz combinada, $\small \textbf{W}_{h} = [\textbf{W}_{xh}; \textbf{W}_{hh}]$. Mais adiante, faremos uso dessa notação também.

A computação das ativações é muito semelhante aos *perceptrons* multicamadas padrão e outros tipos de *RNs* *feedforward*. Para a camada oculta, a entrada líquida, $\small \textbf{z}_ h$ (pré-ativação), é calculada através de uma combinação linear, ou seja, calculamos a soma das multiplicações das matrizes de peso com os vetores correspondentes e somamos a unidade de polarização:
$$
\small \textbf{z}_h^{(t)} = \text{W}_{xh} \textbf{x}^{(t)} + \textbf{W}_{hh}\textbf{h}^{(t-1)} + \textbf{b}_h
$$

Então, as ativações das unidades ocultas na etapa do tempo, $\small t$, são calculadas da seguinte forma:

$$
\textbf{h}^{(t)} = \phi_h (\textbf{z}_h^{(t)}) = \phi_h (\textbf{W}_{xh}\textbf{x}^{(t)} +  \textbf{W}_{hh}\textbf{x}^{(t-1)} + \textbf{b}_h)
$$
 
Aqui, $\small \textbf{b}_h$ é o vetor de polarização para as unidades ocultas e $\phi_h(\cdot)$ é a função de ativação da camada oculta.

Caso você queira usar a matriz de peso concatenada, $\small \textbf{W}_h = [\textbf{W}_{xh}; \textbf{W}_{hh}]$, a fórmula para calcular as unidades ocultas mudará da seguinte forma:

$$
\small \textbf{h}^{(t)} = \phi_h \left ([\textbf{W}_{xh}; \textbf{W}_{hh}] \begin{bmatrix}
\textbf{x}^{(t)}\\ 
\textbf{h}^{t-1}
\end{bmatrix}       
+ \textbf{b}_h          \right )
$$

Uma vez computadas as ativações das unidades ocultas no passo de tempo atual, serão computadas as ativações das unidades de saída, como segue:

$$
\small \textbf{o}^{(t)} = \phi_0 (\textbf{W}_{ho}\textbf{h}^{(t)} + \textbf{b}_0)
$$

Para ajudar a esclarecer ainda mais, a figura a seguir mostra o processo de cálculo dessas ativações com ambas as formulações:

![](imagens\matriz_sequencial.PNG)

> ##### Treinando RNNs usando retropropagação ao longo do tempo (BPTT)
> O algoritmo de aprendizado para RNNs foi introduzido em 1990: Backpropagation Through Time: What It Does and How to Do It (Paul Werbos, Proceedings of IEEE, 78(10): 1550-1560, 1990). A derivada dos gradientes pode ser um pouco complicada, mas a ideia básica é que a perda total, $\small L$, é a soma de todas as funções de perda nos momentos $\small t = 1$ a $\small t = T$:

$$
L = \sum^T_{t=1}L^{(t)}
$$

Como a perda no tempo $\small t$ depende das unidades ocultas em todos os passos de tempo anteriores $\small 1 : t$, o gradiente será calculado da seguinte forma:

$$
\small \dfrac{\partial L^{(t)}}{\partial \textbf{W}_{hh}} = \dfrac{\partial L^{(t)}}{\partial \textbf{o}^{(t)}} \times \dfrac{\partial \textbf{o}^{(t)}}{\partial \textbf{h}^{(t)}} \times \left ( \sum^t_{k=1}\dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}} \times \dfrac{\partial \textbf{h}^{(k)}}{\partial \textbf{W}_{hh}} \right )

$$

Aqui, $\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}}$ é calculado como uma multiplicação de passos de tempo adjacentes:
$$
\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{k}} = \prod ^t_{i=k+1} \dfrac{\partial \textbf{h}^{(i)}}{\partial \textbf{h}^{(i-1)}} 
$$

### Recorrência oculta versus recorrência de saída
Até agora, você viu redes recorrentes nas quais a camada oculta tem a propriedade recorrente. No entanto, observe que existe um modelo alternativo em que a conexão recorrente vem da camada de saída. Nesse caso, as ativações líquidas da camada de saída na etapa de tempo anterior, $\small \textbf{o}^{(t-1)}$, podem ser adicionadas de duas maneiras:
* Para a camada oculta no passo de tempo atual, $\small \textbf{h}^t$ (mostrado na figura a seguir como recorrência de saída para oculta)
* Para a camada de saída no passo de tempo atual, $\small \textbf{o}^{t}$ (mostrado na figura a seguir como recorrência de saída para saída)

![](imagens\recorrencia_saida.PNG)

Conforme mostrado na figura anterior, as diferenças entre essas arquiteturas podem ser vistas claramente nas conexões recorrentes. Seguindo nossa notação, os pesos associados à conexão recorrente serão denotados para a recorrência oculta para oculta por $\small \textbf{W}_{hh}$, para a recorrência saída para oculta por $\small \textbf{W}_{oh}$, e para a recorrência saída para saída por $\small \textbf{W}_{oo}$. Em alguns artigos da literatura, os pesos associados às conexões recorrentes também são denotados por $\small \textbf{W}_{rec}$.

Para ver como isso funciona na prática, vamos calcular manualmente a passagem direta para um desses tipos recorrentes. Usando a *API TensorFlow Keras*, uma camada recorrente pode ser definida por meio da *SimpleRNN*, que é semelhante à recorrência de saída para saída. No código a seguir, criaremos uma camada recorrente do *SimpleRNN* e executaremos uma passagem direta em uma sequência de entrada de comprimento 3 para calcular a saída. Também calcularemos manualmente a passagem direta e compararemos os resultados com os do *SimpleRNN*. Primeiro, vamos criar a camada e atribuir os pesos para nossos cálculos manuais:

In [25]:
import os
import tensorflow as tf
tf.autograph.set_verbosity(0)
os.environ['AUTOGRAPH_VERBOSITY'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

tf.random.set_seed(1)

rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True, 
    return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh shape:', w_xh.shape)
print('W_oo shape:', w_oo.shape)
print('b_h shape:', b_h.shape)

W_xh shape: (5, 2)
W_oo shape: (2, 2)
b_h shape: (2,)


A forma de entrada para esta camada é `(None, None, 5)`, onde a primeira dimensão é a dimensão do lote (usando `None` para tamanho de lote variável), a segunda dimensão corresponde à sequência (usando `None` para o comprimento variável da sequência) e a última dimensão corresponde às características. Observe que definimos `return_sequences=True`, que, para uma sequência de entrada de comprimento 3, resultará na sequência de saída $\small \left \langle \textbf{o}^{(0)},\textbf{o}^{(1)}, \textbf{o}^{(2)} \right \rangle$. Caso contrário, ele retornaria apenas a saída final, $\textbf{o}^{(2)}$.

Agora, vamos chamar a passagem direta no `rnn_layer` e calcular manualmente as saídas em cada passo de tempo e compará-las:

In [26]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)


## output of SimepleRNN:
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('Time step {} =>'.format(t))
    print('   Input           :', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h    
    print('   Hidden          :', ht.numpy())
    
    if t>0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
        
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.numpy())
    print('   SimpleRNN output:'.format(t), output[0][t].numpy())
    print()

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[0.41464037 0.96012145]]
   Output (manual) : [[0.39240566 0.74433106]]
   SimpleRNN output: [0.39240566 0.74433106]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[0.82928073 1.9202429 ]]
   Output (manual) : [[0.80116504 0.99129474]]
   SimpleRNN output: [0.80116504 0.99129474]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[1.243921  2.8803644]]
   Output (manual) : [[0.95468265 0.9993069 ]]
   SimpleRNN output: [0.95468265 0.9993069 ]



Em nosso cálculo direto manual, usamos a função de ativação da tangente hiperbólica (tanh), uma vez que também é usada na `SimpleRNN` (a ativação padrão). Como você pode ver nos resultados impressos, as saídas dos cálculos de encaminhamento manual correspondem exatamente à saída da camada `SimpleRNN` em cada etapa de tempo. Espero que esta tarefa prática tenha esclarecido você sobre os mistérios das redes recorrentes.

### Os desafios de aprender interações de longo prazo

BPTT, que foi brevemente mencionado anteriormente, apresenta alguns novos desafios. Por causa do fator multiplicativo,$\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}}$, ao calcular os gradientes de uma função de perda, surgem os chamados problemas de **gradiente de fuga e explosão**. Esses problemas são explicados pelos exemplos na figura a seguir, que mostra uma RNN com apenas uma unidade oculta para simplificar:

![](imagens\gradiente_fuga_explosao.PNG)


Basicamente, $\small \dfrac{\partial \textbf{h}^{(t)}}{\partial \textbf{h}^{(k)}}$ tem $\small t – k$ multiplicações; portanto, multiplicar o peso, $\small w$, por ele mesmo $\small t – k$ vezes resulta em um fator, $\small w^{t-w}$ . Como resultado, se $\small |w|< 1$, esse fator se torna muito pequeno quando $\small t – k$ é grande. Por outro lado, se o peso da aresta recorrente for $\small |w|> 1$ , então $w^{t-k}$ se torna muito grande quando $\small t – k$ é grande. Observe que $\small t – k$ grande se refere a dependências de longo alcance. Podemos ver que uma solução ingênua para evitar gradientes de fuga ou explosão pode ser alcançada garantindo $\small |w| = 1$.

Na prática, existem pelo menos três soluções para este problema:
* *Gradient clipping*
* *TBPTT*
* *LSTM*

Usando o *Gradient clipping*, especificamos um valor de corte ou limite para os gradientes e atribuímos esse valor de corte aos valores de gradiente que excedem esse valor. Em contraste, o *TBPTT* simplesmente limita ao número de passos de tempo que o sinal pode retropropagar após cada passagem para frente. Por exemplo, mesmo que a sequência tenha 100 elementos ou etapas, podemos retropropagar apenas as 20 etapas de tempo mais recentes.

Embora tanto o *Gradient clipping* quanto o *TBPTT* possam resolver o problema do **gradiente explosivo**, o truncamento limita o número de etapas que o gradiente pode efetivamente retornar e atualizar adequadamente os pesos. Por outro lado, o *LSTM*, projetado em 1997 por *Sepp Hochreiter* e *Jürgen Schmidhuber*, tem tido mais sucesso em eliminar e explodir problemas de gradiente ao modelar dependências de longo alcance por meio de
o uso de células de memória. Vamos discutir *LSTM* com mais detalhes.

### Células de memória de longo prazo

Como afirmado anteriormente, os *LSTMs* foram introduzidos pela primeira vez para superar o problema do gradiente de fuga. O bloco de construção de um *LSTM* é uma célula de memória, que essencialmente representa ou substitui a camada oculta de RNNs padrão.

Em cada célula de memória, há uma aresta recorrente que tem o peso desejável, $\small w = 1$, como discutimos, para superar os problemas de gradiente de fuga e explosão. Os valores associados a essa borda recorrente são chamados coletivamente de estado da célula. A estrutura desdobrada de uma célula *LSTM* moderna é mostrada na figura a seguir:

![](imagens\ltsm.PNG)

Observe que o estado da célula do passo de tempo anterior, $\small \textbf{C}^{(t-1)}$, é modificado para obter o estado da célula no passo de tempo atual, $\small \textbf{C}^{(t)}$, sem ser multiplicado diretamente por nenhum fator de peso. O fluxo de informação nesta célula de memória é controlado por várias unidades de computação (frequentemente chamadas de portas) que serão descritas aqui. Na figura anterior,$\small \bigodot$ refere-se ao produto **elemento-a-elemento** (multiplicação elemento-a-elemento) e $\small \bigoplus$ significa s**oma-elemento** (adição elemento-a-elemento). Além disso, $\small \textbf{x}^{(t)}$ refere-se aos dados de entrada no tempo $\small t$, e $\small \textbf{h}^{(t-1)}$ indica as unidades ocultas no tempo $\small t – 1$. Quatro caixas são indicadas com uma função de ativação, seja a função sigmóide ($\small \sigma$) ou $\small tanh$, e um conjunto de pesos; essas caixas aplicam uma combinação linear realizando multiplicações de matriz-vetor em suas entradas (que são $\small \textbf{h}^{(t-1)}$ e $\small \textbf{x}^{(t)}$). Essas unidades de computação com funções de ativação sigmóides, cujas unidades de saída são passadas por $\small \bigodot$, são chamadas de portas.

Em uma célula *LSTM*, existem três tipos diferentes de portas, que são conhecidas como **porta de esquecimento**, porta de entrada e porta de saída:

* A **porta de esquecimento** ($\small f_t$) permite que a célula de memória redefina o estado da célula sem crescer indefinidamente. Na verdade, o portão de esquecimento decide quais informações podem passar e quais informações devem ser suprimidas. Agora, $\small f_t$ é calculado da seguinte forma:

$$
\small f_t = \sigma(\textbf{W}_{xf}\textbf{x}^{(t)} + \textbf{W}_{hf}\textbf{h}^{(t-1)} + \textbf{b}_f)
$$


* A **porta de entrada** ($\small i_t$) e o **valor candidato** ($\breve{C}_t$) são responsáveis ​​por atualizar o estado da célula. Eles são calculados da seguinte forma:

$$
\small i_t = \sigma ( \textbf{W}_{xi}\textbf{x}^{(t)} +  \textbf{W}_{hi}\textbf{h}^{(t-1)} + \textbf{b}_i)
$$

$$
\small \breve{C}_t = tanh (\textbf{W}_{xc}\textbf{x}^{(t)} + \textbf{W}_{hc}\textbf{h}^{(t-1)} + \textbf{b}_c)
$$

O estado da célula no tempo t é calculado da seguinte forma:

$$
C^{t} = (C^{(t-1)} \odot f_t) \otimes (i_t \odot \breve{C}_t)
$$

A porta de saída ($\small \textbf{o}_t$) decide como atualizar os valores das unidades ocultas:

$$
\small o_t = \sigma (\textbf{W}_{xo}\textbf{x}^{(t)} + \textbf{W}_{ho}\textbf{h}^{(t-1)} + \textbf{b}_o)
$$

Dado isso, as unidades ocultas no passo de tempo atual são calculadas da seguinte forma:

$$
\small \textbf{h}^{(t)} = \textbf{o}_t \odot \: tanh (\textbf{C}^{(t)})
$$

A estrutura de uma célula *LSTM* e seus cálculos subjacentes podem parecer muito complexos e difíceis de implementar. No entanto, a boa notícia é que o *TensorFlow* já implementou tudo em funções otimizadas de wr*apper, o que nos permite definir nossas células *LSTM* de maneira fácil e eficiente. Aplicaremos RNNs e LSTMs a conjuntos de dados do mundo real posteriormente.

> #### Outros modelos RNN avançados
> *LSTMs* fornecem uma abordagem básica para modelar dependências de longo alcance em sequências. No entanto, é importante notar que existem muitas variações de *LSTMs* descritas na literatura (An Empirical Exploration of Recurrent Network Architectures, Rafal Jozefowicz, Wojciech Zaremba e Ilya Sutskever, Proceedings of ICML, 2342-2350, 2015).
>
> Também merece destaque uma abordagem mais recente, *Gated Recurrent Unit* (GRU), proposta em 2014. As GRUs possuem uma arquitetura mais simples que as *LSTMs*; portanto, eles são computacionalmente mais eficientes, enquanto seu desempenho em algumas tarefas, como modelagem de música polifônica, é comparável aos *LSTMs*.

### Implementando RNNs para modelagem de sequência no TensorFlow

Agora que abordamos a teoria subjacente por trás das *RNNs*, estamos prontos para passar para a parte mais prática: implementar *RNNs* no TensorFlow. Durante o restante deste capítulo, aplicaremos *RNNs* a duas tarefas problemáticas comuns:
1. Análise de sentimentos
2. Modelagem de linguagem

Esses dois projetos, que apresentaremos, são fascinantes e envolventes. Assim, em vez de fornecer o código de uma só vez, dividiremos a implementação em várias etapas e discutiremos o código em detalhes.

### **Projeto um – prevendo o sentimento das críticas de filmes do IMDb**

Nesta seção e nas subseções seguintes, implementaremos uma *RNN* multicamada para análise de sentimentos usando uma arquitetura muitos-para-um.

Na próxima seção, implementaremos um *RNN* muitos-para-muitos para uma aplicação de modelagem de linguagem. Embora os exemplos escolhidos sejam propositadamente simples para introduzir os principais conceitos de *RNNs*, a modelagem de linguagem tem uma ampla gama de aplicações interessantes, como a construção de chatbots – dando aos computadores a capacidade de conversar e interagir diretamente com humanos.

### Preparando os dados de revisão do filme
Nas etapas de pré-processamento, criamos um conjunto de dados limpo chamado `movie_data.csv`, que usaremos novamente agora. Primeiro, importaremos os módulos necessários e leremos os dados em um `DataFrame` pandas, da seguinte forma:

In [27]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


Lembre-se de que esse quadro de dados, `df`, consiste em duas colunas, chamadas `'review'` e `'sentiment'`, onde `'review'` contém o texto das resenhas de filmes (os recursos de entrada) e `'sentiment'` representa o rótulo de destino que queremos prever (0 refere-se ao sentimento negativo e 1 refere-se ao sentimento positivo).

O componente de texto dessas resenhas de filmes são sequências de palavras, e o modelo *RNN* classifica cada sequência como uma resenha positiva (1) ou negativa (0). No entanto, antes de podermos alimentar os dados em um modelo *RNN*, precisamos aplicar várias etapas de pré-processamento:
1. Crie um objeto de conjunto de dados do *TensorFlow* e divida-o em partições separadas de treinamento, teste e validação.
2. Identifique as palavras exclusivas no conjunto de dados de treinamento.
3. Mapeie cada palavra exclusiva para um número inteiro exclusivo e codifique o texto da revisão em números inteiros codificados (um índice de cada palavra exclusiva).
4. Divida o conjunto de dados em minilotes como entrada para o modelo.

Vamos prosseguir com a primeira etapa: criar um conjunto de dados do *TensorFlow* a partir deste quadro de dados:

In [28]:
# Passo 1: Criando um Dataset

target = df.pop('sentiment')

ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

## inspection:
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


Agora, podemos dividi-lo em conjuntos de dados de treinamento, teste e validação. Todo o conjunto de dados contém 50.000 exemplos. Manteremos os primeiros 25.000 exemplos para avaliação (conjunto de dados de teste de retenção) e, em seguida, 20.000 exemplos serão usados para treinamento e 5.000 para validação. O código é o seguinte:

In [29]:
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)                 # Separa 25.000 amostras das 50.000
ds_raw_train_valid = ds_raw.skip(25000)          # Pula as 25.000 amostras do conjunto anterior e salva em uma variável 
ds_raw_train = ds_raw_train_valid.take(20000)    # Da variável anterior, vamos pegas 20.000 e salvar numa variável.
ds_raw_valid = ds_raw_train_valid.skip(20000)    # Vamos pular 20.000 amostras e salvar numa variável. Que será somente 5.000 (que sobrou)

Para preparar os dados para entrada em uma RN, precisamos codificá-los em valores numéricos, conforme mencionado nas etapas 2 e 3. Para fazer isso, primeiro encontraremos as palavras exclusivas (*tokens*) no conjunto de dados de treinamento. Embora encontrar *tokens* exclusivos seja um processo para o qual podemos usar conjuntos de dados *Python*, pode ser mais eficiente usar a classe `Counter` do pacote `collections`, que faz parte da biblioteca padrão do *Python*.

No código a seguir, instanciaremos um novo objeto `Counter` (`token_counts`) que coletará as frequências de palavras exclusivas. Observe que neste aplicativo específico (e em contraste com o modelo *bag-of-words*), estamos interessados apenas no conjunto de palavras únicas e não exigiremos a contagem de palavras, que é criada como um produto secundário. Para dividir o texto em palavras (ou *tokens*), o pacote `tensorflow_datasets` fornece uma classe `Tokenizer`.

In [30]:
## Passo 2: Encontrar tokens (palavras) unicas

from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()

token_counts = Counter()

for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)

print('Vocab-size:', len(token_counts))

Vocab-size: 87007


Em seguida, vamos mapear cada palavra única para um inteiro único. Isso pode ser feito manualmente usando um dicionário Python, onde as chaves são os tokens exclusivos (palavras) e o valor associado a cada chave é um inteiro único. No entanto, o pacote `tensorflow_datasets` já fornece uma classe, `TokenTextEncoder`, que podemos usar para criar esse mapeamento e codificar todo o conjunto de dados. Primeiro, criaremos um objeto codificador da classe `TokenTextEncoder` passando os tokens exclusivos (`token_counts` contém os tokens e suas contagens, embora aqui, suas contagens não sejam necessárias, portanto, serão ignoradas). Chamar o método `encoder.encode()` converterá seu texto de entrada em uma lista de valores inteiros:

In [31]:
## Passo 3: Encode tokens únicos (palavras) para inteiros

encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'This is a example!'
print(encoder.encode(example_str))

[232, 9, 35, 1123]


Observe que pode haver alguns *tokens* nos dados de validação ou teste que não estão presentes nos dados de treinamento e, portanto, não estão incluídos no mapeamento. Se tivermos $\small q$ *tokens* (que é o tamanho de `token_counts` passado para o `TokenTextEncoder`, que neste caso é 87.007), todos os *tokens* que não foram vistos antes e, portanto, não estão incluídos em `token_counts`, receberão o inteiro $\small q + 1$ (que será 87.008 no nosso caso). Em outras palavras, o índice $\small q + 1$ é reservado para palavras desconhecidas.

Outro valor reservado é o inteiro 0, que serve como espaço reservado para ajustar o comprimento da sequência. Mais tarde, quando estivermos construindo um modelo *RNN* no *TensorFlow*, consideraremos esses dois espaços reservados, 0 e $\small q + 1$, com mais detalhes.

Podemos usar o método `map()` dos objetos do conjunto de dados para transformar cada texto no conjunto de dados de acordo, assim como aplicaríamos qualquer outra transformação a um conjunto de dados. No entanto, há um pequeno problema: aqui, os dados de texto são incluídos em objetos tensores, que podemos acessar chamando o método `numpy()` em um tensor no modo de execução antecipada. Mas durante as transformações pelo método `map()`, a execução antecipada será desabilitada. Para resolver este problema, podemos definir duas funções. A primeira função tratará os tensores de entrada como se o modo de execução ansioso estivesse habilitado:

In [32]:
## Passo 3-A: defina uma função para a tranformação

def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

Na segunda função, envolveremos a primeira função usando `tf.py_function` para convertê-la em um operador *TensorFlow*, que pode ser usado por meio de seu método `map()`. Este processo de codificação de texto em uma lista de inteiros pode ser realizado usando o seguinte código:

In [33]:
## Passo 3-B: Empacotar a Função encode na Tf. Op

def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
    Tout=(tf.int64, tf.int64))

ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

#  Checando o Shape de alguns exemplos
tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print(f"Tamanho da sequência: {example[0].shape}")

Tamanho da sequência: (24,)
Tamanho da sequência: (179,)
Tamanho da sequência: (262,)
Tamanho da sequência: (535,)
Tamanho da sequência: (130,)


Até agora, convertemos sequências de palavras em sequências de inteiros. No entanto, há um problema que ainda precisamos resolver — as sequências atualmente têm comprimentos diferentes (como mostrado no resultado da execução do código anterior para cinco exemplos escolhidos aleatoriamente). Embora, em geral, as RNNs possam lidar com sequências com comprimentos diferentes, ainda precisamos garantir que todas as sequências em um minilote tenham o mesmo comprimento para armazená-las de forma eficiente em um tensor.

Para dividir um conjunto de dados que possui elementos com formas diferentes em mini-lotes, o *TensorFlow* fornece um método diferente, *padded_batch()* (em vez de `batch()`), que preencherá automaticamente os elementos consecutivos que devem ser combinados em um lote com valores de espaço reservado (0s) para que todas as sequências dentro de um lote tenham a mesma forma. Para ilustrar isso com um exemplo prático, vamos pegar um pequeno subconjunto de tamanho 8 do conjunto de dados de treinamento, `ds_train`, e aplicar o método *padded_batch()* a esse subconjunto com `batch_size=4`. Também imprimiremos os tamanhos dos elementos individuais antes de combiná-los em minilotes, bem como as dimensões dos minilotes resultantes:

In [34]:
## Selecionar um pequeno conjunto de dados:

ds_subset = ds_train.take(8)
for example in ds_subset:
    print(f"Tamanho individual: {example[0].shape}")

Tamanho individual: (119,)
Tamanho individual: (688,)
Tamanho individual: (308,)
Tamanho individual: (204,)
Tamanho individual: (326,)
Tamanho individual: (240,)
Tamanho individual: (127,)
Tamanho individual: (453,)


In [35]:
## Loteando o conjunto de dados

ds_batch = ds_subset.padded_batch(4, padded_shapes=([-1], []))

for batch in ds_batch:
    print(f"Batch dimension: {batch[0].shape}")

Batch dimension: (4, 688)
Batch dimension: (4, 453)


Como você pode observar nas formas de tensor impressas, o número de colunas (ou seja, `.shape[1]`) no primeiro lote é `688`, resultado da combinação dos quatro primeiros exemplos em um único lote e do uso do tamanho máximo desses exemplos. Isso significa que os outros três exemplos neste lote são preenchidos o quanto for necessário para corresponder a esse tamanho.

Da mesma forma, o segundo lote mantém o tamanho máximo de seus quatro exemplos individuais, que é `453`, e preenche os outros exemplos para que seu comprimento seja menor que o comprimento máximo.

Vamos dividir todos os três conjuntos de dados em minilotes com um tamanho de lote de 2:

In [36]:
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))

Agora, os dados estão em um formato adequado para um modelo RNN, que vamos implementar. No próximo tópico, no entanto, discutiremos primeiro a incorporação de recursos, que é uma etapa de pré-processamento opcional, mas altamente recomendada, usada para reduzir a dimensionalidade dos vetores de palavras.

### Incorporando camadas para codificação de frases
Durante a preparação dos dados na etapa anterior, geramos sequências de mesmo comprimento. Os elementos dessas sequências eram números inteiros que correspondiam aos `índices` de palavras únicas. Esses índices de palavras podem ser convertidos em recursos de entrada de várias maneiras diferentes. Uma maneira ingênua é aplicar a codificação *one-hot* para converter os índices em vetores de zeros e uns. Em seguida, cada palavra será mapeada para um vetor cujo tamanho é o número de palavras únicas em todo o conjunto de dados. Dado que o número de palavras únicas (o tamanho do vocabulário) pode ser da ordem de $\small 10^4 - 10^5$, que também será o número de nossos recursos de entrada, um modelo treinado em tais recursos pode sofrer a **maldição da dimensionalidade**. Além disso, esses recursos são muito esparsos, pois todos são zero, exceto um.

Uma abordagem mais elegante é mapear cada palavra para um vetor de tamanho fixo com elementos de valor real (não necessariamente inteiros). Em contraste com os vetores codificados *one-hot*, podemos usar vetores de tamanho finito para representar um número infinito de números reais. (Em teoria, podemos extrair infinitos números reais de um determinado intervalo, por exemplo [–1, 1].)

Essa é a ideia por trás da incorporação (**embedding**), que é uma técnica de aprendizado de recursos que podemos utilizar aqui para aprender automaticamente os recursos importantes para representar as palavras em nosso conjunto de dados. Dado o número de palavras únicas, $\small n_{words}$, podemos selecionar o tamanho dos vetores de incorporação (também conhecido como, a dimensão de incorporação) para ser muito menor que o número de palavras únicas ($\small embedding_{dims} << n_{words}$) para representar todo o vocabulário como recursos de entrada.

As vantagens da incorporação sobre a codificação *one-hot* são as seguintes:
• Uma redução na dimensionalidade do espaço de recursos para diminuir o efeito da maldição da dimensionalidade
• A extração de recursos salientes desde a camada de incorporação em uma RN pode ser otimizada (ou aprendida)

A representação esquemática a seguir mostra como a incorporação funciona mapeando índices de token para uma matriz de incorporação treinável:

![](imagens\embedding.PNG)


Dado um conjunto de tokens de tamanho $\small n + 2$ ($\small n$ é o tamanho do conjunto de *tokens*, mais o índice 0 é reservado para o preenchimento e $\small n + 1$ é para as palavras não presentes no conjunto de *tokens*), uma matriz de incorporação de tamanho $\small (n + 2) \times embedding-dim$ será criado onde cada linha desta matriz representa recursos numéricos associados a um *token*.

Portanto, quando um índice inteiro, $\small i$, for fornecido como entrada para a incorporação, ele procurará a linha correspondente da matriz no índice $\small i$ e retornará os recursos numéricos. A matriz de incorporação serve como camada de entrada para nossos modelos RN.

Na prática, a criação de uma camada de incorporação pode ser feita simplesmente usando `tf.keras.layers.Embedding`. Vamos ver um exemplo onde vamos criar um modelo e adicionar uma camada de embedding, como segue:

In [37]:
from tensorflow.keras.layers import Embedding

model = tf.keras.Sequential()
model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name="embed-layer"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


A entrada para este modelo (camada de incorporação) deve ter a classificação 2 com dimensionalidade $\small batchsize \times input\_length$, onde $\small input\_length$ é o comprimento das seqüências (no caso aqui, definida como 20 através do argumento `input_length`). Por exemplo, uma sequência de entrada no minilote pode ser $\small \left \langle  14,43,52, 1,8,19,67,83,10,7,42,87,56,18,94,17,67,90, 6,39 \right \rangle$, onde cada elemento desta sequência é o índice das palavras únicas. A saída terá dimensionalidade $\small batchsize \times input\_length \times embedding\_dim$, onde $\small embedding\_dim$ é o tamanho dos recursos de incorporação (aqui, definido como 6 via `output_dim`). O outro argumento fornecido à camada de incorporação, `input_dim`, corresponde aos valores inteiros exclusivos que o modelo receberá como entrada (por exemplo, `n + 2`, definido aqui como `100`). Portanto, a matriz de incorporação neste caso tem o tamanho `100 × 6`.

> ##### Lidando com comprimentos de sequência variáveis
> Observe que o argumento `input_length` não é necessário e podemos usar `None` para casos em que os comprimentos das sequências de entrada variam.

### Construindo um modelo *RNN*
Agora estamos prontos para construir um modelo *RNN*. Usando a classe Keras `Sequential`, podemos combinar a camada de incorporação (*embedding*), as camadas recorrentes da *RNN* e as camadas não recorrentes totalmente conectadas. Para as camadas recorrentes, podemos usar qualquer uma das seguintes implementações:
* `SimpleRNN`: uma camada *RNN* regular, ou seja, uma camada recorrente totalmente conectada
* `LSTM`: uma *RNN* de memória de curto prazo longo, que é útil para capturar as dependências de longo prazo
* `GRU`: uma camada recorrente com uma unidade recorrente fechada, como alternativa aos `LSTMs`

Para ver como um modelo *RNN* multicamada pode ser construído usando uma dessas camadas recorrentes, no exemplo a seguir, criaremos um modelo *RNN*, começando com uma camada de incorporação com `input_dim=1000` e `output_dim=32`. Em seguida, serão adicionadas duas camadas recorrentes do tipo `SimpleRNN`. Por fim, adicionaremos uma camada totalmente conectada não recorrente como camada de saída, que retornará um único valor de saída como previsão:

In [38]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Usando uma RNN com LTSM 
from tensorflow.keras.layers import LSTM
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [40]:
## Um exemplo com uma camada GRU
from tensorflow.keras.layers import GRU

model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          320000    
                                                                 
 gru (GRU)                   (None, None, 32)          6336      
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 332,705
Trainable params: 332,705
Non-trainable params: 0
_________________________________________________________________


Como você pode ver, construir um modelo *RNN* usando essas camadas recorrentes é bastante simples. Na próxima subseção, voltaremos à nossa tarefa de análise de sentimentos e construiremos um modelo *RNN* para resolver isso.

### Construindo um modelo *RNN* para a tarefa de análise de sentimento

Como temos sequências muito longas, usaremos uma camada *LSTM* para contabilizar os efeitos de longo prazo. Além disso, colocaremos a camada *LSTM* dentro de um wrapper `Bidirectional`, que fará com que as camadas recorrentes passem pelas sequências de entrada de ambas as direções, do início ao fim, bem como no sentido inverso:

In [41]:
embedding_dim = 20
vocab_size = len(token_counts) + 2

tf.random.set_seed(1)

## Construindo o modelo 
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'), 

    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

## Compilando e treinando:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

## Avaliando os dados
test_results= bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1740180   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


KeyboardInterrupt: 

Após treinar este modelo por 10 épocas, a avaliação nos dados de teste mostra **81%** de precisão. (Observe que esse resultado não é o melhor quando comparado aos métodos de última geração usados no conjunto de dados do IMDb. O objetivo era simplesmente mostrar como o RNN funciona.)

>##### Mais sobre o RNN bidirecional
> O wrapper `Bidirectional` faz duas passagens em cada sequência de entrada: uma passagem para frente e uma passagem reversa ou para trás (observe que isso não deve ser confundido com as passagens para frente e para trás no contexto de retropropagação). Os resultados dessas passagens para frente e para trás serão concatenados por padrão. Mas se você quiser mudar esse comportamento, você pode definir o argumento `merge_mode` para `'sum'` (para soma), `'mul'` (para multiplicar os resultados das duas passagens), `'ave'` (para tirar a média das duas) , `'concat'` (que é o padrão) ou `None`, que retorna os dois tensores em uma lista.

Também podemos tentar outros tipos de camadas recorrentes, como `SimpleRNN`. No entanto, como se vê, um modelo construído com camadas recorrentes regulares não será capaz de alcançar um bom desempenho preditivo (mesmo nos dados de treinamento). Por exemplo, se você tentar substituir a camada *LSTM* bidirecional no código anterior por uma camada `SimpleRNN` unidirecional e treinar o modelo em sequências completas, poderá observar que a perda nem diminuirá durante o treinamento. **A razão é que as sequências neste conjunto de dados são muito longas**, portanto, um modelo com uma camada `SimpleRNN` não pode aprender as dependências de longo prazo e pode sofrer com problemas de **gradiente de fuga ou explosão**.

Para obter um desempenho preditivo razoável neste conjunto de dados usando um `SimpleRNN`, podemos truncar as sequências. Além disso, utilizando nosso "conhecimento de domínio", podemos supor que os últimos parágrafos de uma crítica de filme podem conter a maioria das informações sobre seu sentimento. Portanto, podemos nos concentrar apenas na última parte de cada revisão. Para fazer isso, vamos definir uma função auxiliar, `preprocess_datasets()`, para combinar as etapas de pré-processamento `2-4`. Um argumento opcional para esta função é `max_seq_length`, que determina quantos *tokens* de cada revisão devem ser usados. Por exemplo, se definirmos `max_seq_length=100` e um comentário tiver mais de 100 *tokens*, apenas os últimos 100 *tokens* serão usados. Se `max_seq_length` for definido como `None`, as sequências de comprimento total serão usadas como antes. Tentar valores diferentes para `max_seq_length` nos dará mais informações sobre a capacidade de diferentes modelos de RNN para lidar com sequências longas.

O código para a função `preprocess_datasets()` é o seguinte:

In [ ]:
from collections import Counter
def preprocess_datasets(
    ds_raw_train, 
    ds_raw_valid, 
    ds_raw_test,
    max_seq_length=None,
    batch_size=32):
    
    ## Passo 1: (Já feito => Cria o DataSet)
    ## Passo 2: (Encontra os tokens únicos (palavras)
    tokenizer = tfds.deprecated.text.Tokenizer()
    token_counts = Counter()

    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens = tokens[-max_seq_length:]
        token_counts.update(tokens)

    print('Vocab-size:', len(token_counts))


    ## Passo 3: Encode os textos
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_seq_length is not None:
            encoded_text = encoded_text[-max_seq_length:]
        return encoded_text, label

    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label], 
                              Tout=(tf.int64, tf.int64))

    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)

    ## Passo 4: Lotea o DataSet
    train_data = ds_train.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    valid_data = ds_valid.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    test_data = ds_test.padded_batch(
        batch_size, padded_shapes=([-1],[]))

    return (train_data, valid_data, 
            test_data, len(token_counts))

Em seguida, vamos definir outra função auxiliar, `build_rnn_model()`, para construir modelos com diferentes arquiteturas de forma mais conveniente:

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):

    tf.random.set_seed(1)

    # Constrói o modelo
    model = tf.keras.Sequential()
    
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers-1)
            
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='simprnn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_units, 
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
        
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-'+recurrent_layer.name)
            
        model.add(recurrent_layer)

    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

Agora, usando essas duas funções auxiliares bastante gerais, mas convenientes, podemos comparar prontamente diferentes modelos RNN com diferentes comprimentos de sequência de entrada. Como exemplo, no código a seguir, tentaremos um modelo com uma única camada recorrente do tipo `SimpleRNN` enquanto truncamos as sequências para um comprimento máximo de 100 *tokens*:

In [ ]:
from tensorflow.keras.layers import Bidirectional


batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
    ds_raw_train, ds_raw_valid, ds_raw_test, 
    max_seq_length=max_seq_length, 
    batch_size=batch_size
)


vocab_size = n + 2

rnn_model = build_rnn_model(
    embedding_dim, vocab_size,
    recurrent_type='SimpleRNN', 
    n_recurrent_units=64,
    n_recurrent_layers=1,
    bidirectional=True)

rnn_model.summary()

Vocab-size: 58063
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1161300   
                                                                 
 bidir-simprnn-layer-0 (Bidi  (None, 128)              10880     
 rectional)                                                      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])


history = rnn_model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs=10)

Epoch 1/10
625/625 [==============================] - 173s 275ms/step - loss: 0.7013 - accuracy: 0.5030 - val_loss: 0.6968 - val_accuracy: 0.5040
Epoch 2/10
625/625 [==============================] - 177s 282ms/step - loss: 0.6858 - accuracy: 0.5490 - val_loss: 0.6403 - val_accuracy: 0.6286
Epoch 3/10
625/625 [==============================] - 176s 281ms/step - loss: 0.4813 - accuracy: 0.7696 - val_loss: 0.4830 - val_accuracy: 0.7886
Epoch 4/10
625/625 [==============================] - 176s 282ms/step - loss: 0.2786 - accuracy: 0.8864 - val_loss: 0.5534 - val_accuracy: 0.7630
Epoch 5/10
625/625 [==============================] - 176s 282ms/step - loss: 0.1624 - accuracy: 0.9371 - val_loss: 0.6887 - val_accuracy: 0.7518
Epoch 6/10
625/625 [==============================] - 176s 281ms/step - loss: 0.0504 - accuracy: 0.9829 - val_loss: 1.0899 - val_accuracy: 0.7372
Epoch 7/10
625/625 [==============================] - 178s 284ms/step - loss: 0.0285 - accuracy: 0.9915 - val_loss: 1.3421 -

Por exemplo, truncar as sequências para 100 tokens e usar uma camada `SimpleRNN` bidirecional resultou em **80%** de precisão de classificação. Embora a previsão seja um pouco menor quando comparada ao modelo *LSTM* bidirecional anterior (**85,15%** de precisão no conjunto de dados de teste), o desempenho nessas sequências truncadas é muito melhor do que o desempenho que poderíamos alcançar com um `SimpleRNN` em resenhas de filmes completos. Como exercício opcional, você pode verificar isso usando as duas funções auxiliares que já definimos. Experimente com `max_seq_length=None` e defina o argumento bidirecional dentro da função auxiliar `build_rnn_model()` como `False`.

### **Projeto dois – modelagem de linguagem em nível de caractere no *TensorFlow***

A modelagem de linguagem é uma aplicatição fascinante que permite que as máquinas executem tarefas relacionadas à linguagem humana, como gerar frases em inglês.

No modelo que construiremos agora, a entrada é um documento de texto, e nosso objetivo é desenvolver um modelo que possa gerar um novo texto com estilo semelhante ao documento de entrada. Exemplos de tal entrada são um livro ou um programa de computador em uma linguagem de programação específica.

Na modelagem de linguagem em nível de caractere, a entrada é dividida em uma sequência de caracteres que são inseridos em nossa rede, um caractere por vez. A rede processará cada novo personagem em conjunto com a memória dos personagens vistos anteriormente para prever o próximo. A figura a seguir mostra um exemplo de modelagem de linguagem em nível de caractere (observe que EOS significa *End of Sequence* (fim de sequência)):
![](imagens\modelagem_nivel_caractere.PNG)

Podemos dividir essa implementação em três etapas separadas:
1. preparar os dados, construir o modelo RNN e realizar previsão, e
2. amostragem do próximo caractere para gerar novo texto.

### Pré-processamento do conjunto de dados
Nesta seção, prepararemos os dados para modelagem de linguagem em nível de caractere.

Para obter os dados de entrada, visite o site do Project Gutenberg em https://www.gutenberg.org/, que fornece milhares de e-books gratuitos. Para o nosso exemplo, você pode baixar o livro A Ilha Misteriosa, de Júlio Verne (publicado em 1874) em formato de texto simples de http://www.gutenberg.org/files/1268/1268-0.txt.

Depois de baixar o conjunto de dados, podemos lê-lo em uma sessão do Python como texto simples. Usando o código a seguir, vamos ler o texto diretamente do arquivo baixado e remover partes do início e do fim (estes contêm certas descrições do projeto Gutenberg). Em seguida, criaremos uma variável *Python*, `char_set`, que representa o conjunto de caracteres `unique` observados neste texto:

In [ ]:
import numpy as np

## Lendo o arquivo e processando o texto
arquivo = "https://www.gutenberg.org/files/1268/1268-0.txt"

with open (arquivo, encoding='utf8') as fp:
    text = fp.read()

start_indx = text.find("THE MISTERIOUS ISLAND")
end_indx = text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]
char_set = set(text)
print(f"Tamanho total: {len(text)}")

OSError: [Errno 22] Invalid argument: 'https://www.gutenberg.org/files/1268/1268-0.txt'

In [43]:
import os
import requests
url = "https://www.gutenberg.org/files/1268/1268-0.txt"
resposta = requests.get(url=url)
endereco = os.path.basename(url.split('?')[0])
if resposta.status_code == requests.codes.OK:
    with open(endereco, 'wb') as arquivo:
        arquivo.write(resposta.content)
    print('Download finalizado. Arquivo salvo em: {}'.format(endereco))

with open('1268-0.txt', 'r', encoding='utf8') as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print()

text = text[start_indx:end_indx]
char_set = set(text)
print('Tamanho total:', len(text))
print('Caracteres únicos:', len(char_set))

Download finalizado. Arquivo salvo em: 1268-0.txt

Tamanho total: 1112350
Caracteres únicos: 80


Após o download e pré-processamento do texto, temos uma sequência composta por 1.112.350 caracteres no total e 80 caracteres únicos. No entanto, a maioria das bibliotecas NN e implementações RNN não podem lidar com dados de entrada em formato de *string*, e é por isso que temos que converter o texto em um formato numérico. Para fazer isso, vamos criar um dicionário Python simples que mapeia cada caractere para um inteiro, `char2int`. Também precisaremos de um mapeamento reverso para converter os resultados do nosso modelo de volta em texto.

Embora o inverso possa ser feito usando um dicionário que associa chaves inteiras a valores de caracteres, usar um array *NumPy* e indexar o array para mapear índices para esses caracteres exclusivos é mais eficiente. A figura a seguir mostra um exemplo de conversão de caracteres em inteiros e o inverso para as palavras "Hello" e "world":

![](imagens\mapeamento_reverso.PNG)

A construção do dicionário para mapear caracteres para inteiros e o mapeamento reverso por meio da indexação de um array *NumPy*, conforme mostrado na figura anterior, é a seguinte:

In [54]:
chars_sorted = sorted(char_set)                        # Organizando o Conjunto ÚNICO da dados
char2int = {ch:i for i, ch in enumerate(chars_sorted)} # Criando os índices para o Conjunto organizado em um Dict
char_array = np.array(chars_sorted)                    # Criando um Array com o Conjunto ordenado.

text_encoded = np.array(                               # Aplicando o mapeamento no texto de Júlio Verne
                        [char2int[ch] for ch in text],
                        dtype=np.int32   
                       )              
print(f'Text Encode Shape: {text_encoded.shape}')       

print(text[:15], '== Encoding ==>', text_encoded[:15]) # Aplicando o Encode nos primeiros 15 caracteres
print(text_encoded[15:21], '== Reverse ==>',''.join(char_array[text_encoded[15:21]])) # Aplicando o decode no caracteres ISLAND

Text Encode Shape: (1112350,)
THE MYSTERIOUS  == Encoding ==> [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28] == Reverse ==> ISLAND


A matriz NumPy `text_encoded` contém os valores codificados para todos os caracteres no texto. Agora, vamos criar um conjunto de dados do *TensorFlow* a partir deste array:

In [56]:
import tensorflow as tf

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print(f'{ex.numpy()} ===> {char_array[ex.numpy()]}')

44 ===> T
32 ===> H
29 ===> E
1 ===>  
37 ===> M


Até agora, criamos um objeto Dataset iterável para obter caracteres na ordem em que aparecem no texto. Agora, vamos dar um passo atrás e olhar para o quadro geral do que estamos tentando fazer. Para a tarefa de geração de texto, podemos formular o problema como uma tarefa de classificação.

Suponha que tenhamos um conjunto de sequências de caracteres de texto incompletos, conforme mostrado na figura a seguir:

![](imagens\sequencias_targets.PNG)

Na figura anterior, podemos considerar as sequências mostradas na caixa à esquerda como sendo a entrada. Para gerar um novo texto, nosso objetivo é projetar um modelo que possa prever o **próximo caractere** de uma determinada sequência de entrada, onde a sequência de entrada representa um texto incompleto. Por exemplo, depois de ver `"Deep Learn"`, o modelo deve prever `"i"` como o próximo caractere. Dado que temos 80 caracteres únicos, este problema torna-se uma tarefa de classificação multiclasse.

Começando com uma sequência de comprimento 1 (ou seja, uma única letra), podemos gerar iterativamente novo texto com base nessa abordagem de classificação multiclasse, conforme ilustrado na figura a seguir:

![](imagens\sequencias_classificacao.PNG)


Para implementar a tarefa de geração de texto no *TensorFlow*, primeiro vamos cortar o comprimento da sequência para 40. Isso significa que o tensor de entrada, x, consiste em 40 tokens. **Na prática, o comprimento da sequência impacta na qualidade do texto gerado**. Sequências mais longas podem resultar em frases mais significativas. Para sequências mais curtas, no entanto, o modelo pode se concentrar em capturar palavras individuais corretamente, ignorando o contexto na maior parte.

Embora sequências mais longas geralmente resultem em sentenças mais significativas, como mencionado, para sequências longas, o modelo RNN terá problemas para capturar dependências de longo prazo. Assim, na prática, encontrar um ponto ideal e um bom valor para o comprimento da sequência é um problema de otimização de hiperparâmetros, que temos que avaliar empiricamente. Aqui, vamos escolher 40, pois oferece uma boa troca.

Como você pode ver na figura anterior, as entradas, $\small \textbf{x}$, e os destinos, $\small \textbf{y}$, são deslocados por um caractere. Assim, dividiremos o texto em pedaços de tamanho 41: os primeiros 40 caracteres formarão a sequência de entrada, $\small \textbf{x}$, e os últimos 40 elementos formarão a sequência alvo, $\small \textbf{y}$.

Já armazenamos todo o texto codificado em sua ordem original em um objeto `Dataset`, `ds_text_encoded`. Usando as técnicas relativas à transformação de conjuntos de dados que já abordamos, você pode pensar em uma maneira de obter a entrada, $\small \textbf{x}$, e o destino, $\small \textbf{y}$, como foi mostrado na figura anterior? A resposta é muito simples: primeiro usaremos o método `batch()` para criar blocos de texto com 41 caracteres cada. Isso significa que definiremos `batch_size=41`. Vamos nos livrar ainda mais do último lote se for menor que 41 caracteres. Como resultado, o novo conjunto de dados fragmentado, denominado `ds_chunks`, sempre conterá sequências de tamanho 41. Os fragmentos de 41 caracteres serão usados ​​para construir a sequência $\small \textbf{x}$ (ou seja, a entrada), bem como a sequência $\small \textbf{y}$ (que é, o alvo), ambos terão 40 elementos. Por exemplo, a sequência x consistirá dos elementos com índices [0, 1,...,39]. Além disso, como a sequência $\small \textbf{y}$ será deslocada de uma posição em relação a $\small \textbf{x}$, seus índices correspondentes serão [1, 2,..., 40]. Em seguida, aplicaremos uma função de transformação usando o método `map()` para separar as sequências $\small \textbf{x}$ e $\small \textbf{y}$ de acordo:

In [69]:
seq_length = 40
chunk_size = seq_length + 1

ds_chuncks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

# Definir uma Função que divide X e Y

def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

# Aplicar o map() em todo conjunto com a função

ds_sequences = ds_chuncks.map(split_input_target)

Vamos ver alguns exemplos deste dataset transformado:

In [70]:
for example in ds_sequences.take(2):
    print(f"  Input  (x): {repr(''.join(char_array[example[0].numpy()]))}" )
    print(f"  Target (y): {repr(''.join(char_array[example[1].numpy()]))}" )
    print()

  Input  (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
  Target (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

  Input  (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
  Target (y): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



Finalmente, a última etapa na preparação do conjunto de dados é dividir esse conjunto de dados em minilotes. Durante a primeira etapa de pré-processamento para dividir o conjunto de dados em lotes, criamos pedaços de frases. Cada pedaço representa uma frase, que corresponde a um exemplo de treinamento. Agora, vamos embaralhar os exemplos de treinamento e dividir as entradas em mini-lotes novamente; no entanto, desta vez, cada lote conterá vários exemplos de treinamento:

In [74]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

ds.element_spec

(TensorSpec(shape=(None, 40), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 40), dtype=tf.int32, name=None))

### Construindo um modelo RNN em nível de personagem
Agora que o conjunto de dados está pronto, a construção do modelo será relativamente simples. Para reutilização de código, escreveremos uma função, `build_model`, que define um modelo RNN usando a classe Keras `Sequential`. Então, podemos especificar os parâmetros de treinamento e chamar essa função para obter um modelo RNN:

In [75]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 256)         20480     
                                                                 
 lstm_2 (LSTM)               (None, None, 512)         1574912   
                                                                 
 dense_5 (Dense)             (None, None, 80)          41040     
                                                                 
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


Observe que a camada *LSTM* neste modelo tem o formato de saída (None, None, 512), o que significa que a saída de *LSTM* é de *rank* 3. A primeira dimensão representa o número de lotes, a segunda dimensão o comprimento da sequência de saída e a última dimensão corresponde ao número de unidades ocultas. A razão para ter a saída de *rank* 3 da camada *LSTM* é porque especificamos `return_sequences=True` ao definir nossa camada *LSTM*. Uma camada totalmente conectada (`Dense`) recebe a saída da célula *LSTM* e calcula os logits para cada elemento das sequências de saída. Como resultado, a saída final do modelo também será um tensor de *rank* 3.
Além disso, especificamos `activation=None` para a camada final totalmente conectada. A razão para isso é que precisaremos ter os logits como saídas do modelo para que possamos amostrar as previsões do modelo para gerar um novo texto. Chegaremos a esta parte de amostragem mais tarde. Por enquanto, vamos treinar o modelo:

In [76]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=20)

Epoch 1/20
424/424 [==============================] - 13s 28ms/step - loss: 2.3232
Epoch 2/20
424/424 [==============================] - 12s 29ms/step - loss: 1.7520
Epoch 3/20
424/424 [==============================] - 12s 28ms/step - loss: 1.5413
Epoch 4/20
424/424 [==============================] - 12s 28ms/step - loss: 1.4213
Epoch 5/20
424/424 [==============================] - 12s 28ms/step - loss: 1.3473
Epoch 6/20
424/424 [==============================] - 12s 28ms/step - loss: 1.2971
Epoch 7/20
424/424 [==============================] - 12s 28ms/step - loss: 1.2596
Epoch 8/20
424/424 [==============================] - 12s 28ms/step - loss: 1.2296
Epoch 9/20
424/424 [==============================] - 12s 28ms/step - loss: 1.2040
Epoch 10/20
424/424 [==============================] - 12s 28ms/step - loss: 1.1827
Epoch 11/20
424/424 [==============================] - 12s 28ms/step - loss: 1.1635
Epoch 12/20
424/424 [==============================] - 12s 28ms/step - loss: 1.1464
E

Agora, podemos avaliar o modelo para gerar um novo texto, começando com uma determinada *string* curta. Na próxima seção, definiremos uma função para avaliar o modelo treinado.

### Fase de avaliação – geração de novas passagens de texto

O modelo RNN que treinamos na seção anterior retorna os logits de tamanho 80 para cada caractere único. Esses logits podem ser facilmente convertidos em probabilidades, por meio da função *softmax*, de que um determinado caractere seja encontrado como o próximo caractere. Para prever o próximo caractere na sequência, basta selecionar o elemento com o valor logit máximo, o que equivale a selecionar o caractere com maior probabilidade.

No entanto, em vez de sempre selecionar o caractere com a maior probabilidade, **queremos amostrar (aleatoriamente) das saídas**; caso contrário, o modelo sempre produzirá **o mesmo texto**. O *TensorFlow* já fornece uma função, `tf.random.categorical()`, que podemos usar para extrair amostras aleatórias de uma distribuição categórica. Para ver como isso funciona, vamos gerar algumas amostras aleatórias de três categorias [0, 1, 2], com logits de entrada [1, 1, 1].

In [78]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 1.0]]

print(f"Probabilidades: {tf.math.softmax(logits=logits).numpy()[0]}")

Probabilidades: [0.33333334 0.33333334 0.33333334]


In [79]:
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())

array([[1, 2, 0, 1, 0, 1, 1, 2, 1, 1]], dtype=int64)


Como você pode ver, com os logits dados, as categorias têm as mesmas probabilidades (ou seja, categorias equiprováveis). Portanto, se usarmos um tamanho de amostra grande ($\small num\_samples \rightarrow \infty)$, esperaríamos que o número de ocorrências de cada categoria chegasse a $\small \approx \dfrac{1}{3}$ do tamanho da amostra. Se alterarmos os logits para [1, 1, 3], esperaríamos observar mais ocorrências para a categoria 2 (quando um número muito grande de exemplos é extraído dessa distribuição):

In [80]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]

print(f"Probabilidades: {tf.math.softmax(logits=logits).numpy()[0]}")

Probabilidades: [0.10650698 0.10650698 0.78698605]


In [81]:
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())

array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]], dtype=int64)


Usando `tf.random.categorical`, podemos gerar exemplos com base nos logits calculados pelo nosso modelo. Definimos uma função, `sample()`, que recebe uma *string* inicial curta, `starting_str`, e gera uma nova *string*, `generated_str`, que é inicialmente definida como a *string* de entrada. Em seguida, uma *string* de tamanho `max_input_length` é retirada do final de `generated_str` e codificada em uma sequência de inteiros, `encoded_input`. O `encoded_input` é passado para o modelo RNN para calcular os logits. Observe que a saída do modelo RNN é uma sequência de logits com o mesmo comprimento da sequência de entrada, pois especificamos `return_sequences=True` para a última camada recorrente de nosso modelo RNN. Portanto, cada elemento na saída do modelo RNN representa os logits (aqui, um vetor de tamanho 80, que é o número total de caracteres) para o próximo caractere após observar a sequência de entrada pelo modelo.

Aqui, usamos apenas o último elemento dos logits de saída (ou seja, $\small \textbf{o}^{(T)}$ ), que é passado para a função `tf.random.categorical()` para gerar uma nova amostra. Essa nova amostra é convertida em um caractere, que é anexado ao final da *string* gerada, `generated_text`, aumentando seu comprimento em 1. Em seguida, esse processo é repetido, pegando o último número de caracteres `max_input_length` do final do `generated_str`, e usando isso para gerar um novo caractere até que o comprimento da *string* gerada atinja o valor desejado. O processo de consumir a sequência gerada como entrada para geração de novos elementos é chamado de `auto-regression` (**auto-regressão**).

> #### Retornando sequências como saída
> 
> Você pode se perguntar por que usamos `return_sequences=True` quando usamos apenas o último caractere para amostrar um novo caractere e ignoramos o restante da saída. Embora essa pergunta faça todo o sentido, você não deve esquecer que usamos toda a sequência de saída para treinamento. A perda é calculada com base em cada previsão na saída e não apenas na última.

O código para a função `sample()` é o seguinte:

In [85]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

Vamos agora gerar algum texto novo:

In [83]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island was eggled from the passage of a power part. It was cleaning the
rejoice, though but a good one of these plantagio, which they wished to go to sighter the animal, indergounacial bay, and
capricious apparatus. Cyrus Harding
thought
of confinence had recies of the downs of the colonists, without finding again lors. Cyrus Harding bad
of leavings which he sainful
arrangements.”

At seals chosen suddenly.

The engineer’s head, and therefore, caused there without wishing. The time was scanty informed


Como você pode ver, o modelo gera principalmente palavras corretas e, em alguns casos, as frases são **parcialmente significativas**. Você pode ajustar ainda mais os parâmetros de treinamento, como o comprimento das sequências de entrada para treinamento, a arquitetura do modelo e os parâmetros de amostragem (como `max_input_length`).

Além disso, para controlar a previsibilidade das amostras geradas (ou seja, gerar texto seguindo os padrões aprendidos do texto de treinamento versus adicionar mais aleatoriedade), os logits calculados pelo modelo RNN podem ser dimensionados antes de serem passados para `tf.random.categorical()` para amostragem. O fator de escala, $\small \alpha$ , pode ser interpretado como o inverso da temperatura na física. Temperaturas mais altas resultam em mais aleatoriedade versus comportamento mais previsível em temperaturas mais baixas. Ao escalonar os logits com $\small \alpha < 1$, as probabilidades calculadas pela função `softmax` se tornam mais uniformes, conforme mostrado no código a seguir:

In [91]:
logits = np.array([[1.0, 1.0, 3.0]])
print(f"Probabilidades antes de dimensionar:           {tf.math.softmax(logits).numpy()[0]}")
print(f"Probabilidades depois de dimensionar com 0.5:  {tf.math.softmax(0.5*logits).numpy()[0]}")
print(f"Probabilidades depois de dimensionar com 0.1:  {tf.math.softmax(0.1*logits).numpy()[0]}")

Probabilidades antes de dimensionar:           [0.10650698 0.10650698 0.78698604]
Probabilidades depois de dimensionar com 0.5:  [0.21194156 0.21194156 0.57611688]
Probabilidades depois de dimensionar com 0.1:  [0.31042377 0.31042377 0.37915245]


Como você pode ver, dimensionar os logits por $\small \alpha = 0.1$ resulta em probabilidades quase uniformes [0.31, 0.31, 0.38]. Agora, podemos comparar o texto gerado com $\small \alpha = 2.0$ e $\small \alpha = 0.5$, conforme mostrado nos seguintes pontos:

* $\small \alpha = 2.0 \rightarrow mais\: previsível$

In [92]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', scale_factor=2.0))

The island was again the stranger, and I say our ‘Bonadventure.”

“Then the reporter was called the bank of the rock, and the brig had been a possible to start the state of the crater.

“And you, Mr. Spilett, and these things were a few minutes the destruction of the grotto, and the settlers had the engineer and his companions left the stranger fell on the shore, and the reporter and the reporter and Pencroft, and you will be a good part of the volcano.”

“Well, my boy, but at the same time against the co


* $\small \alpha = 0.5 \rightarrow mais\: aleatório$

In [93]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', scale_factor=0.5))

The island had egbly sleepliggt and the Happy, Herbert! ciff swelland,
myrifrt-lumble,” then, tharked Cune--lan,.. Vade. he
keb-darh tak;
unforwing our Islan her a Farxisy neither sixaptific ode, or carbly voice at the verage cold Pencroft.

occoppeded shallibre!”; terrishod and push, was
already nized it in af excetill reliam lithudencuces,
of quelusis,
Cyrus Harding Awar and a frise massidy relun. Had Nabual:-illy oursquise, yem in any equal froster. Lay nwers-fall, with three. Would have sombling A9uri


Os resultados mostram que dimensionar os logits com $\small \alpha = 0.5$ (aumentando a temperatura) gera mais texto aleatório. Há uma compensação entre a novidade do texto gerado e sua correção.

Nesta seção, trabalhamos com geração de texto em nível de caractere, que é uma tarefa de modelagem de sequência a sequência (seq2seq). Embora este exemplo possa não ser muito útil por si só, é fácil pensar em várias aplicações úteis para esses tipos de modelos; por exemplo, um modelo RNN semelhante pode ser treinado como um *chatbot* para auxiliar os usuários com consultas simples.